In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm

# Generate synthetic normal sensor data
np.random.seed(42)
n = 100
timestamps = pd.date_range("2025-01-01", periods=n, freq="s")
true_values = np.random.normal(loc=25, scale=0.5, size=n)

# Introduce corruption (spikes and missing values)
corrupted_values = true_values.copy()
corrupted_values[10] += 8    # spike
corrupted_values[25] -= 7    # drop
corrupted_values[40] = np.nan  # missing
corrupted_values[60] += 6
corrupted_values[75] = np.nan

# MCMC Correction using a simple normal model
corrected_values = corrupted_values.copy()

with pm.Model() as model:
    mu = pm.Normal("mu", mu=25, sigma=1)
    sigma = pm.HalfNormal("sigma", sigma=1)

    observed = pm.Normal("obs", mu=mu, sigma=sigma, observed=corrupted_values[~np.isnan(corrupted_values)])

    trace = pm.sample(1000, tune=500, chains=2, target_accept=0.95, return_inferencedata=False, progressbar=False)

    # Impute missing values using posterior mean
    corrected_mean = trace["mu"].mean()

    # Replace corrupted/missing manually identified
    for i in [10, 25, 40, 60, 75]:
        corrected_values[i] = corrected_mean

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(timestamps, true_values, label="True (Original)", linestyle="--", color="green", alpha=0.7)
plt.plot(timestamps, corrupted_values, label="Corrupted", color="red", linewidth=1.5)
plt.plot(timestamps, corrected_values, label="MCMC Corrected", color="blue", linewidth=1.5)
plt.xlabel("Time")
plt.ylabel("Sensor Reading (°C)")
plt.title("MCMC Correction of Corrupted Time Series Data")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("/home/rbakyayita/Documents/MScIS-thesis/mscis/figures/MCMC_Simulation.png", dpi=300)
plt.close()

print("MCMC correction plot saved as '/home/rbakyayita/Documents/MScIS-thesis/mscis/figures/MCMC_Simulation.png'")


ModuleNotFoundError: No module named 'pymc3'

In [10]:
pip install pymc3


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai